In [1]:
import urllib
import bs4
import re
import pandas as pd
from urllib import request
import datetime
import time
import re
import codecs

In [9]:
## création d"une liste de dates 
liste_dates=[]
j = datetime.date(2020, 11, 30)
while j >= datetime.date(2020, 10, 1):
    liste_dates.append(str(j.strftime("%Y/%m/%d")))
    j = j + datetime.timedelta(days=-1)
print(len(liste_dates))
liste_dates[:1]

61


['2020/11/30']

In [10]:
# création de 2 listes l'une contient les articles, l'autre le jour de publication.
liste_titres = []
liste_jour_publication = []
liste_categorie =[]
liste_auteurs =[]
df_row = pd.DataFrame()
for d in liste_dates:
    ## exploration des pages internet en ajoutant la date à un début d'url
    status = 999
    compt = 1
    while status != 404:
        try:
            page_explorée = "https://www.lexpress.fr/archives/" + str(d)
            req_1 = urllib.request.Request(page_explorée,headers={'User-Agent': 'Mozilla/5.0'})
            request_text = request.urlopen(req_1).read()
            page = bs4.BeautifulSoup(request_text, "lxml")
            for titre in page.findAll("h3", {'class' : "title_mu"}):
                t = titre.get_text()
                liste_titres.append(t)
            for categorie in page.findAll("span", {'class' : "topic"}):
                cat = categorie.get_text()
                if len(cat)==0:
                    cat = ['']
                liste_categorie.append(cat)
                liste_jour_publication.append(d)   
            for group in page.findAll("div", {'class':"groups search_results"}):
                for url in group.findAll("a"):
                    url_article = url['href']
                    url_2 ="https:"+ url_article
                    req_2 = urllib.request.Request(url_2,headers={'User-Agent': 'Mozilla/5.0'})
                    request_text2 = request.urlopen(req_2).read()
                    page_article = bs4.BeautifulSoup(request_text2, "lxml")
                    tit=''
                    s=''
                    for div in page_article.findAll("div",{'class':"author__name"}):
                        for sp in div.findAll('span'):
                            for s in sp.findAll('span')[0]:
                                liste_auteurs.append(s) # ajout de la liste d'auteur associé à l'article    

            compt+=1  
        except Exception as e:
            status = 404
len(liste_titres)

3014

In [11]:
print(len(liste_titres))
print(len(liste_categorie))
print(len(liste_jour_publication))
print(len(liste_auteurs))

3014
3014
3014
55


In [12]:
df = pd.DataFrame({ 'titre':liste_titres,'jour_publication':liste_jour_publication,'categorie':liste_categorie})
df

,titre,jour_publication,categorie
0,"Algérie: le président Tebboune va rentrer ""dan...",2020/11/30,Monde
1,"Affaire des ""écoutes"": Sarkozy dénonce des ""in...",2020/11/30,Société
2,Vaccin Covid-19: les autorités sanitaires pose...,2020/11/30,Société
3,La Cour suprême désarçonnée par la dernière re...,2020/11/30,Monde
4,"Scientifique tué: une opération ""complexe"", l'...",2020/11/30,Monde
...,...,...,...
3009,Vers un 4e mandat pour le président LR du Séna...,2020/10/01,Actualité
3010,"Le fonds Ardian, ""soutenu"" par Suez, ""intéress...",2020/10/01,Société
3011,Charlie Hebdo raconte ses 50 ans de lutte pour...,2020/10/01,Actualité
3012,Comment fonctionnent les pétitions citoyennes ...,2020/10/01,Politique


In [13]:
df.to_csv('titres_Lexpress_oct_nov2020.csv')